This Anime image classification was created by Vignesh SS for Databyte Inductions. Adding further training data, more layers of CNN, with more iterations will even produce a higher accuracy i.e. greater than 73.64% received by this model, but at the cost of higher computation.

In [2]:
!pip install bing-image-downloader

In [3]:
!mkdir Input_images

from bing_image_downloader import downloader

In [4]:
characters = [
    "Iron Man", "Spider-Man", "Captain America", "Hulk", "Thor",
    "Black Widow", "Doctor Strange", "Black Panther", "Captain Marvel", "Ant-Man",
    "Naruto", "Goku", "Luffy", "Saitama", "Deku",
    "Sasuke", "Vegeta", "Ichigo", "Natsu", "Eren"
]

In [5]:
# Function to download images for each character
def download_images(character_list, limit=60):     #Using 60 images instead of 30
    for character in character_list:
        print(f"Downloading images for {character}...")
        downloader.download(character + " figurine", limit=limit, output_dir='Input_images/Figurines', adult_filter_off=True)
        print(f"Downloaded images for {character}")

In [ ]:
# Download images for all characters
download_images(characters)

Our dataset is downloaded now for 20 characters with 60 images for each character making it to function well.

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [8]:
# Path to the dataset
data_dir = 'Input_images/Figurines'

In [9]:
# Create ImageDataGenerator for data augmentation and preprocessing

datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values to [0,1]
    rotation_range=10,           # Rotate images by up to 10 degrees
    width_shift_range=0.1,       # Shift the width of images by up to 10%
    height_shift_range=0.1,      # Shift the height of images by up to 10%
    shear_range=0.2,             # Randomly apply shearing
    zoom_range=0.2,              # Randomly zoom into images
    horizontal_flip=True,        # Randomly flip images horizontally
    brightness_range=[0.8, 1.2], # Adjust brightness
    #contrast_range=[0.8, 1.2],   # Adjust contrast
    validation_split=0.2         # Split for validation
)


In [10]:
# Create training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,           # Number of images to process in a batch
    class_mode='categorical',# Use categorical labels
    subset='training',        # Specify this is the training subset
    shuffle=True            # Shuffle the training data
)


Found 960 images belonging to 20 classes.


In [11]:

# Create validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,           # Number of images to process in a batch
    class_mode='categorical',# Use categorical labels
    subset='validation',     # Specify this is the validation subset
    shuffle=True            # Shuffle the validation data ( Additionally added)
)

Found 239 images belonging to 20 classes.


Now lets create our CNN model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max')
]

In [16]:
# Train the model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
30/30 [==============================] - 41s 1s/step - loss: 7.3205 - accuracy: 0.1375 - val_loss: 5.3529 - val_accuracy: 0.0491


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
30/30 [==============================] - 30s 1s/step - loss: 3.4736 - accuracy: 0.1729 - val_loss: 11.2332 - val_accuracy: 0.0580
Epoch 3/100
30/30 [==============================] - 28s 954ms/step - loss: 2.8712 - accuracy: 0.2250 - val_loss: 11.6885 - val_accuracy: 0.0446
Epoch 4/100
30/30 [==============================] - 29s 975ms/step - loss: 2.5808 - accuracy: 0.2479 - val_loss: 17.7373 - val_accuracy: 0.0625
Epoch 5/100
30/30 [==============================] - 32s 1s/step - loss: 2.4248 - accuracy: 0.2948 - val_loss: 15.2973 - val_accuracy: 0.0491
Epoch 6/100
30/30 [==============================] - 33s 1s/step - loss: 2.3081 - accuracy: 0.3187 - val_loss: 19.7741 - val_accuracy: 0.0670
Epoch 7/100
30/30 [==============================] - 32s 1s/step - loss: 2.2192 - accuracy: 0.3708 - val_loss: 18.8077 - val_accuracy: 0.0982
Epoch 8/100
30/30 [==============================] - 32s 1s/step - loss: 2.1975 - accuracy: 0.3531 - val_loss: 13.2889 - val_accuracy: 0.0804


In [17]:
# Load the best model
model = tf.keras.models.load_model('best_model.h5')

In [18]:

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


8/8 [==============================] - 8s 909ms/step - loss: 1.4218 - accuracy: 0.7364
Validation Accuracy: 73.64%


In [19]:
# Function to predict the character from a new image
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[class_idx]



In [20]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=60aaae7d8a9c042a5b53a434195c7af4e73e79b3d41011db910f8e84846b5a78
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [21]:
import wget
image_url = "https://ichef.bbci.co.uk/images/ic/1200x675/p09t1hg0.jpg"
image_filename = "my_favourite.jpg"
wget.download(image_url, image_filename)

'my_favourite.jpg'

In [22]:

# Test the prediction function
image_path = "my_favourite.jpg"
print(f'The given image is: {predict_image(image_path)}')


1/1 [==============================] - 1s 576ms/step
The given image is: Captain America figurine


In [23]:

# Print the class indices to verify correct labeling
print("Class indices:", train_generator.class_indices)

# Evaluate the model and print the accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Check the class indices
class_labels = list(train_generator.class_indices.keys())
print(f'Class labels: {class_labels}')

Class indices: {'Ant-Man figurine': 0, 'Black Panther figurine': 1, 'Black Widow figurine': 2, 'Captain America figurine': 3, 'Captain Marvel figurine': 4, 'Deku figurine': 5, 'Doctor Strange figurine': 6, 'Eren figurine': 7, 'Goku figurine': 8, 'Hulk figurine': 9, 'Ichigo figurine': 10, 'Iron Man figurine': 11, 'Luffy figurine': 12, 'Naruto figurine': 13, 'Natsu figurine': 14, 'Saitama figurine': 15, 'Sasuke figurine': 16, 'Spider-Man figurine': 17, 'Thor figurine': 18, 'Vegeta figurine': 19}
8/8 [==============================] - 8s 1s/step - loss: 1.2339 - accuracy: 0.7238
Validation Accuracy: 72.38%
Class labels: ['Ant-Man figurine', 'Black Panther figurine', 'Black Widow figurine', 'Captain America figurine', 'Captain Marvel figurine', 'Deku figurine', 'Doctor Strange figurine', 'Eren figurine', 'Goku figurine', 'Hulk figurine', 'Ichigo figurine', 'Iron Man figurine', 'Luffy figurine', 'Naruto figurine', 'Natsu figurine', 'Saitama figurine', 'Sasuke figurine', 'Spider-Man figurine

As this model was overfiiting the training set previously, increasing the training set to 60 images for a character and training for more number of iterations has increased its accuracy from 59% to 73%.